In [1]:
''' RNN 신경망 구성하기 '''

import tensorflow as tf
import numpy as np

# 전체 알파벳 정보를 저장합니다.
char_arr = ['a', 'b', 'c', 'd', 'e', 'f',
            'g', 'h', 'i', 'j', 'k', 'l',
            'm', 'n', 'o', 'p', 'q', 'r',
            's', 't', 'u', 'v', 'w', 'x',
            'y', 'z']

# a부터 z까지 인덱스를 붙여 저장합니다.
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 학습에 사용할 단어를 배열로 저장합니다.
seq_data = ['such', 'case', 'dare', 'duel', 'late', 'carp',
            'hall', 'java', 'fell', 'none', 'dark', 'doom',
            'dark', 'dark', 'sqrt', 'them', 'them', 'here']

def make_batch(seq_data):
    input_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 단어의 처음 세 글자의 알파벳 인덱스를 구한 배열을 생성합니다.
        input = [num_dic[n] for n in seq[:-1]]
        # 마지막 글자의 알파벳 인덱스를 구합니다.
        target = num_dic[seq[-1]]
        # 입력값을 원-핫 인코딩 방식으로 변환합니다.
        input_batch.append(np.eye(dic_len)[input])
        # 출력값은 원-핫 인코딩이 아닌 인덱스 그대로 이용합니다.
        target_batch.append(target)
        
    return input_batch, target_batch


In [2]:
''' RNN 신경망 학습 모델 구성하기 '''

# RNN 학습 오류 방지를 위해 그래프 리셋 함수를 호출합니다.
tf.reset_default_graph()

# 모든 알파벳을 배경으로 3글자를 단계적으로 학습합니다.
X = tf.placeholder(tf.float32, [None, 3, dic_len])
# 출력 값으로는 인덱스 그 자체를 이용합니다.
Y = tf.placeholder(tf.int32, [None])

# 은닉층의 크기는 256입니다.
W = tf.Variable(tf.random_normal([256, dic_len]))
b = tf.Variable(tf.random_normal([dic_len]))

# 과적합 현상을 방지하며 두 개의 RNN 셀을 이용합니다.
cell1 = tf.nn.rnn_cell.BasicLSTMCell(256)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.7)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(256)

# 두 셀을 조합하여 심층 순환 신경망을 구축합니다.
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 출력층을 구축합니다.
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

# 인덱스 값을 그대로 결과로 사용할 수 있도록 비용 함수를 적용해 최적화합니다.
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [3]:
''' RNN 신경망 모델 학습하기 '''

# 그래프의 동작을 위해 세션을 초기화합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 입력 값과 출력 값으로 단어를 분리합니다.
input_batch, target_batch = make_batch(seq_data)

# 10번 반복하여 학습합니다.
for epoch in range(100):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    
    print('반복:', '%04d' % (epoch + 1), '비용:', '{:.4f}'.format(loss))

print('학습 완료!')

반복: 0001 비용: 3.5806
반복: 0002 비용: 1.9336
반복: 0003 비용: 1.5777
반복: 0004 비용: 1.7899
반복: 0005 비용: 1.4384
반복: 0006 비용: 0.9035
반복: 0007 비용: 0.4830
반복: 0008 비용: 0.4160
반복: 0009 비용: 0.3685
반복: 0010 비용: 0.3434
반복: 0011 비용: 0.3259
반복: 0012 비용: 0.1758
반복: 0013 비용: 0.1939
반복: 0014 비용: 0.1842
반복: 0015 비용: 0.2459
반복: 0016 비용: 0.1581
반복: 0017 비용: 0.1205
반복: 0018 비용: 0.1478
반복: 0019 비용: 0.1340
반복: 0020 비용: 0.1642
반복: 0021 비용: 0.1244
반복: 0022 비용: 0.1687
반복: 0023 비용: 0.1275
반복: 0024 비용: 0.0862
반복: 0025 비용: 0.1553
반복: 0026 비용: 0.1347
반복: 0027 비용: 0.1085
반복: 0028 비용: 0.1755
반복: 0029 비용: 0.1471
반복: 0030 비용: 0.1732
반복: 0031 비용: 0.1069
반복: 0032 비용: 0.1314
반복: 0033 비용: 0.0865
반복: 0034 비용: 0.1384
반복: 0035 비용: 0.1887
반복: 0036 비용: 0.1037
반복: 0037 비용: 0.2041
반복: 0038 비용: 0.1435
반복: 0039 비용: 0.2380
반복: 0040 비용: 0.2671
반복: 0041 비용: 0.1563
반복: 0042 비용: 0.1584
반복: 0043 비용: 0.1103
반복: 0044 비용: 0.1792
반복: 0045 비용: 0.1173
반복: 0046 비용: 0.1527
반복: 0047 비용: 0.1207
반복: 0048 비용: 0.2257
반복: 0049 비용: 0.1033
반복: 0050 비용: 0.1992


In [4]:
''' RNN 모델 결과 확인하기 '''

prediction = tf.cast(tf.argmax(model, 1), tf.int32)
# 출력값 Y는 인덱스 그대로 사용합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

# 학습에 사용한 단어를 넣고 예측 모델을 동작시킵니다.
input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

predict_words = []
for index, value in enumerate(seq_data):
    # 예측 모델이 예측한 마지막 단어를 저장합니다.
    last_char = char_arr[predict[index]]
    # 기존의 세 단어와 예측 단어를 합쳐 출력합니다.
    predict_words.append(value[:3] + last_char)
    
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)

입력값: ['suc ', 'cas ', 'dar ', 'due ', 'lat ', 'car ', 'hal ', 'jav ', 'fel ', 'non ', 'dar ', 'doo ', 'dar ', 'dar ', 'sqr ', 'the ', 'the ', 'her ']
예측값: ['such', 'case', 'dark', 'duel', 'late', 'carp', 'hall', 'java', 'fell', 'none', 'dark', 'doom', 'dare', 'dark', 'sqrt', 'them', 'them', 'here']
정확도: 0.8888889
